# Pipeline Presets Demo

This notebook demonstrates the built-in pipeline presets for common training workflows:

1. **supervised_train** / **supervised_val** - Standard ImageNet training/validation
2. **simclr** - SimCLR two-view SSL
3. **byol** - BYOL two-view SSL (asymmetric augmentation)
4. **multicrop** - Multi-crop SSL (DINO, iBOT)

Each preset returns a `pipelines` dict ready for `SlipstreamLoader(pipelines=...)`.

In [ ]:
LITDATA_VAL_PATH = "s3://visionlab-datasets/imagenet1k/pre-processed/s256-l512-jpgbytes-q100-streaming/val/"

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from slipstream import SlipstreamDataset, SlipstreamLoader

dataset = SlipstreamDataset(
    remote_dir=LITDATA_VAL_PATH,
    decode_images=False,
)
print(f"Dataset: {len(dataset):,} samples")

In [ ]:
def show_batch(images, title="", nrow=8, figsize=None):
    """Display a grid of images from a [B, C, H, W] float tensor."""
    if isinstance(images, torch.Tensor):
        images = images.cpu()
    n = min(len(images), nrow * 2)
    ncols = min(n, nrow)
    nrows = (n + ncols - 1) // ncols
    if figsize is None:
        figsize = (ncols * 1.8, nrows * 1.8)
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    if nrows == 1:
        axes = [axes] if ncols == 1 else list(axes)
        axes = [axes]
    for i in range(nrows):
        for j in range(ncols):
            idx = i * ncols + j
            ax = axes[i][j]
            if idx < n:
                img = images[idx].permute(1, 2, 0).numpy()
                # Clamp normalized images for display
                if img.min() < 0 or img.max() > 1:
                    img = np.clip(img * 0.229 + 0.485, 0, 1)  # Approximate denormalize
                ax.imshow(img)
            ax.axis('off')
    if title:
        fig.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 1. Supervised Training Pipeline

`supervised_train` provides the standard ImageNet training transform:
- `DecodeRandomResizedCrop(224)` - Random crop + resize
- `ToTorchImage` - Convert to float [0,1], device transfer
- `Normalize` - ImageNet mean/std normalization

In [ ]:
from slipstream.pipelines import supervised_train

# Inspect the pipeline structure
pipe = supervised_train(size=224, device='cpu', normalize=True)
print("supervised_train pipeline:")
for i, stage in enumerate(pipe['image']):
    print(f"  {i}: {stage}")

In [ ]:
loader = SlipstreamLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    seed=42,
    pipelines=supervised_train(size=224, device='cpu', normalize=True),
    exclude_fields=['path'],
)

batch = next(iter(loader))
print(f"Batch keys: {list(batch.keys())}")
print(f"Image shape: {batch['image'].shape}, dtype: {batch['image'].dtype}")
print(f"Image range: [{batch['image'].min():.3f}, {batch['image'].max():.3f}] (normalized)")
loader.shutdown()

In [ ]:
show_batch(batch['image'], 'supervised_train: RandomResizedCrop + Normalize')

## 2. Supervised Validation Pipeline

`supervised_val` provides the standard ImageNet validation transform:
- `DecodeResizeCrop(256 -> 224)` - Resize shortest edge to 256, center crop to 224
- `ToTorchImage` + `Normalize`

In [ ]:
from slipstream.pipelines import supervised_val

# Inspect the pipeline
pipe = supervised_val(size=224, device='cpu')
print("supervised_val pipeline:")
for i, stage in enumerate(pipe['image']):
    print(f"  {i}: {stage}")

In [ ]:
loader = SlipstreamLoader(
    dataset,
    batch_size=16,
    shuffle=False,  # No shuffle for validation
    pipelines=supervised_val(size=224, device='cpu'),
    exclude_fields=['path'],
)

batch = next(iter(loader))
print(f"Image shape: {batch['image'].shape}")
print(f"Image range: [{batch['image'].min():.3f}, {batch['image'].max():.3f}]")
loader.shutdown()

In [ ]:
show_batch(batch['image'], 'supervised_val: Resize(256) + CenterCrop(224) + Normalize')

## 3. SimCLR Two-View Pipeline

`simclr` returns a multi-view pipeline with two independently augmented views:
- Both views: RandomResizedCrop + HFlip + ColorJitter + Grayscale + GaussianBlur
- View 1 only: Solarization

Returns `{'image': [[view0_stages], [view1_stages]]}`.

In [ ]:
from slipstream.pipelines import simclr

pipe = simclr(size=224, seed=42, device='cpu', normalize=False)
print("simclr pipeline structure:")
print(f"  Type: {type(pipe['image'])}")
print(f"  Number of views: {len(pipe['image'])}")
print("\nView 0 stages:")
for i, stage in enumerate(pipe['image'][0]):
    print(f"  {i}: {type(stage).__name__}")
print("\nView 1 stages:")
for i, stage in enumerate(pipe['image'][1]):
    print(f"  {i}: {type(stage).__name__}")

In [ ]:
loader = SlipstreamLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    seed=42,
    pipelines=simclr(size=224, seed=42, device='cpu', normalize=False),
    exclude_fields=['path'],
)

batch = next(iter(loader))
print(f"Batch keys: {list(batch.keys())}")
print(f"batch['image'] is a list of {len(batch['image'])} tensors")
print(f"View 0: {batch['image'][0].shape}, dtype: {batch['image'][0].dtype}")
print(f"View 1: {batch['image'][1].shape}, dtype: {batch['image'][1].dtype}")
loader.shutdown()

In [ ]:
# Show both views side-by-side
n = min(6, batch['image'][0].shape[0])
fig, axes = plt.subplots(2, n, figsize=(n * 2.2, 4.6))
for i in range(n):
    for row in range(2):
        img = batch['image'][row][i].permute(1, 2, 0).numpy()
        axes[row][i].imshow(np.clip(img, 0, 1))
        axes[row][i].axis('off')
    if i == 0:
        axes[0][i].set_ylabel('View 0', fontsize=11)
        axes[1][i].set_ylabel('View 1\n(+Solar)', fontsize=11)
fig.suptitle('SimCLR: Two views with independent augmentations', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 4. BYOL Two-View Pipeline (Asymmetric)

`byol` provides BYOL's asymmetric augmentation:
- **View 0 (online)**: Strong blur (p=1.0), no solarization
- **View 1 (target)**: Weak blur (p=0.1), solarization (p=0.2)

In [ ]:
from slipstream.pipelines import byol

loader = SlipstreamLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    seed=42,
    pipelines=byol(size=224, seed=42, device='cpu', normalize=False),
    exclude_fields=['path'],
)

batch = next(iter(loader))
print(f"View 0 (online): {batch['image'][0].shape}")
print(f"View 1 (target): {batch['image'][1].shape}")
loader.shutdown()

In [ ]:
n = min(6, batch['image'][0].shape[0])
fig, axes = plt.subplots(2, n, figsize=(n * 2.2, 4.6))
for i in range(n):
    for row in range(2):
        img = batch['image'][row][i].permute(1, 2, 0).numpy()
        axes[row][i].imshow(np.clip(img, 0, 1))
        axes[row][i].axis('off')
    if i == 0:
        axes[0][i].set_ylabel('Online\n(strong blur)', fontsize=10)
        axes[1][i].set_ylabel('Target\n(weak blur+solar)', fontsize=10)
fig.suptitle('BYOL: Asymmetric augmentation (online vs target)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Multi-Crop Pipeline (DINO / iBOT)

`multicrop` provides the DINO/iBOT multi-crop strategy:
- N global crops (large scale range, e.g., 224px)
- M local crops (small scale range, e.g., 96px)

Uses `DecodeMultiRandomResizedCrop` (decode-once fusion) + `MultiCropPipeline` (per-crop augmentations).

In [ ]:
from slipstream.pipelines import multicrop

pipe = multicrop(
    global_crops=2, local_crops=4,
    global_size=224, local_size=96,
    seed=42, device='cpu', normalize=False,
)
print("multicrop pipeline structure:")
print(f"  Image pipeline stages: {len(pipe['image'])}")
for i, stage in enumerate(pipe['image']):
    print(f"  {i}: {type(stage).__name__}")

In [ ]:
loader = SlipstreamLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    seed=42,
    pipelines=multicrop(
        global_crops=2, local_crops=4,
        global_size=224, local_size=96,
        seed=42, device='cpu', normalize=False,
    ),
    exclude_fields=['path'],
)

batch = next(iter(loader))
crop_keys = [k for k in batch.keys() if k.startswith(('global_', 'local_'))]
print(f"Batch keys: {list(batch.keys())}")
print(f"\nCrop outputs:")
for k in crop_keys:
    print(f"  {k}: {batch[k].shape}, dtype: {batch[k].dtype}")
loader.shutdown()

In [ ]:
# Show all 6 crops for first 6 images
n_images = min(6, batch['global_0'].shape[0])
crop_names = ['global_0', 'global_1', 'local_0', 'local_1', 'local_2', 'local_3']
n_crops = len(crop_names)

fig, axes = plt.subplots(n_crops, n_images, figsize=(n_images * 2.5, n_crops * 2.5))
for row, name in enumerate(crop_names):
    crop = batch[name]
    for col in range(n_images):
        img = crop[col].permute(1, 2, 0).numpy()
        axes[row][col].imshow(np.clip(img, 0, 1))
        axes[row][col].axis('off')
        if col == 0:
            size = crop.shape[-1]
            axes[row][col].set_ylabel(f"{name}\n({size}px)", fontsize=10)
        if row == 0:
            axes[row][col].set_title(f"Image {col}", fontsize=10)
fig.suptitle('Multi-crop (DINO-style): 2 global (224px) + 4 local (96px) with augmentations',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Custom Pipeline Example

You can also build custom pipelines by importing components directly:

In [ ]:
from slipstream.decoders import DecodeRandomResizedCrop, DecodeCenterCrop
from slipstream.transforms import (
    ToTorchImage, Normalize, RandomHorizontalFlip,
    RandomColorJitterHSV, RandomGrayscale,
    IMAGENET_MEAN, IMAGENET_STD,
)

# Custom training pipeline with grayscale augmentation
custom_train = {
    'image': [
        DecodeRandomResizedCrop(224, scale=(0.2, 1.0)),
        ToTorchImage(device='cpu'),
        RandomHorizontalFlip(p=0.5),
        RandomGrayscale(p=0.5),  # 50% grayscale
        Normalize(IMAGENET_MEAN, IMAGENET_STD),
    ]
}

loader = SlipstreamLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    seed=42,
    pipelines=custom_train,
    exclude_fields=['path'],
)

batch = next(iter(loader))
loader.shutdown()

show_batch(batch['image'], 'Custom pipeline: RRC + HFlip + 50% Grayscale + Normalize')

## Summary

| Preset | Use Case | Output Format |
|--------|----------|---------------|
| `supervised_train` | ImageNet training | `batch['image']` tensor |
| `supervised_val` | ImageNet validation | `batch['image']` tensor |
| `simclr` | SimCLR two-view SSL | `batch['image']` list of 2 tensors |
| `byol` | BYOL asymmetric SSL | `batch['image']` list of 2 tensors |
| `multicrop` | DINO/iBOT multi-crop | `batch['global_0']`, `batch['local_0']`, etc. |

All presets support:
- `size` - output crop size
- `seed` - base seed for reproducibility (None for random)
- `device` - target device for GPU augmentations
- `normalize` - whether to apply ImageNet normalization